### Initialize

In [30]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['firstName', 'lastName', 'age']]
            df.rename(columns={'firstName': 'First Name',
                      'lastName': 'Last Name', 'age': 'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


def execute_search_queries(search_objects): 
    queries = [ so.dict(exclude_none=True) for so in search_objects ]
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=queries)
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['entityDesc']]
            df.rename(columns={'entityDesc': 'Description'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())



### Search for a Person where the Location of its is in London
--> It is searched for a person where a location of a person is related to London

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relLocation', 'location', 'city']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='London'))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for a Person where the case is about money laundering
--> It is searched for a person who is involved in money laundering 

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy A: Money Laundering', isPhrase=True))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for persons where the case contains the words "Kennedy Money Laundering"

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy Money Laundering', doEshEscaping=True))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))

## Search for Persons where the case is about money laundering and the leads includes "payment"
--> This is an enhancment of the query before. It is searched for the payments of the money laundering

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relLeads', 'lead', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy J Cash payment', doEshEscaping=True)) 
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for the time range of the payments when the person did the money laundering
--> This is an enhancement of the query before. It is searched for the time ranges when the payments were done

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relLeads', 'lead', 'entityTime']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[2013-01-01 2019-12-12]'))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for a case where the actvity was a phone intercept between J Kennedy and B Ramage

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Cases'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relActivity', 'activity', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Phone intercept: J Kennedy - B Ramage', isPhrase=True))  # String search with spaces and semi colons not possible
        ]))


r = requests.post(
    f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'caseStartDate', 'caseSummary', 'priority']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time', 'caseStartDate': 'Start Date', 'caseSummary': 'Summary', 'priority': 'Priority'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


## Search for the objects P Lomez of an activity

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Activity'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relObject', 'object', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='P lomez*', doEshEscaping=True))  # String search with spaces and semi colons not possible
        ]))


r = requests.post(
    f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


## Search for persons who has an incident with "Threat of violence" at the location "Dingley Apartments, Braddon"

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'incident', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Threat of violence')), 
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'incident', 'location']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Dingley Apartments, Braddon', doEshEscaping=True)
            )
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for witness who reported the incident "Threat of violence Kennedy Gomez"

In [ ]:

so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'relTypeCode', 'description']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Witness'))
                ,  
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident','incident','entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Threat of violence Kennedy Gomez', doEshEscaping=True))    

        ]))

execute_search_query(so.dict(exclude_none=True))

## Search for persons who has relationships to each other #1

In [47]:

so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.OR,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relPerson', 'person' ,'firstName']), #'relLocation','location','relPerson','person',
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Alex')),
            esh.Comparison(
                property=esh.Property(
                    property=['relObject','object','relPerson','person','firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Alex'))  

        ]))

execute_search_query(so.dict(exclude_none=True))

Total count: 3


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Alex,Kennedy,57
2,Phillipe,Lomez,28


## Search for persons who has relationships to each other #2

In [22]:

so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.OR,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relPerson', 'person' ,'firstName']), #'relLocation','location','relPerson','person',
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Joseph')),
            esh.Comparison(
                property=esh.Property(
                    property=['relLeads','lead','relPerson','person','firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Joseph')),
             esh.Comparison(
                property=esh.Property(
                    property=['relObject','object','relPerson','person','firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Joseph')),
             esh.Comparison(
                property=esh.Property(
                    property=['relIncident','incident','relPerson','person','firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Joseph'))    

        ]))

execute_search_query(so.dict(exclude_none=True))

Total count: 7


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Alex,Kennedy,57
2,Phillipe,Lomez,28
3,Jerry,Roberts,37
4,Hilda,Friedel,51
5,Rosario,Gomez,42
6,Rogerio,Gomez,42


## Search for two persons and figure out how they are connected to each other (several conditions)

In [43]:

search_objects = [
    esh.EshObject(
        count=True,
        # top=1,
        scope=['Cases'],
        searchQueryFilter=esh.Expression(
            operator=esh.LogicalOperator.AND,
            items=[
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Joseph')),
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Alex'))

            ])),
    esh.EshObject(
        count=True,
        # top=1,
        scope=['Leads'],
        searchQueryFilter=esh.Expression(
            operator=esh.LogicalOperator.AND,
            items=[
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Joseph')),
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Alex'))

            ])),
    esh.EshObject(
        count=True,
        # top=1,
        scope=['Object'],
        searchQueryFilter=esh.Expression(
            operator=esh.LogicalOperator.AND,
            items=[
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Joseph')),
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Alex'))

            ])),
    esh.EshObject(
        count=True,
        # top=1,
        scope=['Incidents'],
        searchQueryFilter=esh.Expression(
            operator=esh.LogicalOperator.AND,
            items=[
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Joseph')),
                esh.Comparison(
                    property=esh.Property(
                        property=['relPerson', 'person', 'firstName']),
                    operator=esh.ComparisonOperator.Search,
                    value=esh.StringValue(value='Alex'))

            ]))                        
            ]

execute_search_queries(search_objects)


Total count: 1


,Description
0,Kennedy A: Money Laundering


## Search for two persons and figure out how they are connected to each other (simple condition)

In [46]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Incidents'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'firstName']), 
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Jerry')),
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Joseph'))  

        ]))

# execute_search_query(so.dict(exclude_none=True))
r = requests.post(
    f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[so.dict(exclude_none=True)])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'position','location']]
        df.rename(columns={'entityDesc': 'Description',
                    'entityTime': 'Time', 'position': 'Position', 'location':'Location'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())

Total count: 1


,Description,Time,Position,Location
0,Assault: J Kennedy on J Roberts,2011-10-11 15:15:00,"{'type': 'Point', 'coordinates': [149.0666687,...","Kennedy Cleaning, Greenway"
